In [1]:
import matplotlib
%matplotlib qt  
#%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy



/tmp/ipykernel_19068/361147393.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


ModuleNotFoundError: No module named 'sklearn'

# Fix missing python packages

In [2]:
# Install missing packages



# then rerun the top/first cell again to see if packages are detected



# Get required raw data for our task from dropbox

In [3]:
# go to google drive download the files from the binarization folder
# make sure you know the location of the .zip file

import webbrowser
url = 'https://drive.google.com/drive/folders/1fiyQdL55S3kkAa0EcUj8fpo50o3zhQcy?usp=sharing'
webbrowser.open(url)



True

In [4]:
# unzip the binarization_data.zip file 
# you should be able to doubleclick on the file and unzip it using windows zip


# Run [Ca] data binarization algorithms

In [5]:
# set input where our unzipped files are 
# this directory should have all the numpy files outputed from suite2p:
#  e.g.  F.npy
#        Fnew.npy 
#        spks..npy ...
# 

data_dir = '/home/cat/Downloads/binarization_data/'
#data_dir = r'C:\...'



In [6]:
# initialize calcium object and load suite2p data
c = calcium.Calcium()                     # initialize a calcium object that will hold our data and process it
c.verbose = True                          # outputs additional information during processing: True / False
c.recompute_binarization = True           # recomputes binarization and other processing steps; False: loads from previous saved locations
c.data_dir = data_dir


In [7]:
# we load the output from suite2p 
c.load_suite2p()                          


  Fluorescence data loading information
         sample rate:  30 hz
         self.F (fluorescence):  (1232, 55740)
         self.Fneu (neuropile):  (1232, 55740)
         self.iscell (Suite2p cell classifier output):  (2131, 2)
              of which number of good cells:  (1232,)
         self.spks (deconnvoved spikes):  (1232, 55740)
         self.stat (footprints structure):  (1232,)
         mean std over all cells :  21.5


In [8]:
# set flags to save matlab and python data
c.save_python = True          # save output as .npz file to be loaded later by us using python
c.save_matlab = False         # save output as .mat file if you wish to load the output later using Matlab


In [9]:
#  First set of parameters
#  - this is the threshold for calling a calcium trace event high enough to be considered a real event - rather than random fluctuation
#  - the higher we set this threshold, the less events we get (more precise)
#  - the lower we set this threshold, the more events (but more noise/random events)
#
#  Suggest to start at default of 2.5 and increase it if data is very noisy and getting too many noise-events
c.min_thresh_std_onphase = 2.5      # set the minimum thrshold for onphase detection; defatul 2.5
c.min_thresh_std_upphase = 2.5      # set the minimum thershold for uppohase detection; default: 2.5

# onphase - is the detection of entire calcium events passing the threshold
# upphase - is the detection of only the initial "UP" or rising phases of the calcium events
#     more will be seen below in the visualizations

#
# TASK: play around with these values and visualize single cells and rasters below



In [10]:
#  Second parameters to filter data
#  - we filter the [Ca] traces which smooths the data significantly more 
#  - we use a "low pass butter worth filter" with a "high_cutoff" value 
#  - note: higher cutoff gives more calcium events but adds noise
#          lower cutoff gives less calcium events but increases accuracy of detection

#
c.high_cutoff = 0.5               # default is 0.5hz but 1.0 hz can also work

# TASK: play around with this filter and see how it affects the final rasters


In [11]:
# Third set of parameters to remove artifcats from the baseline of the [Ca] traces
# - this value is used to fit a polynomial to the [ca] traces and then remove it 

#
c.detrend_model_order = 1 # 1-5 polynomial fit
                          # polynomial of degree 1 = line
                          # polynomial of degree 2 = parabola ...



In [12]:
# Finally we run the binarization step

# 
c.binarize_fluorescence()


  Binarization parameters: 
        low pass filter low cuttoff:  0.5 hz
        oasis_thresh_prefilter:  15
        min_thresh_std_oasis:  0.1
        min_thresh_std_onphase:  2.5
        min_thresh_std_upphase:  2.5
        min_width_event_onphase:  15
        min_width_event_upphase:  7
        min_width_event_oasis:  2
        min_event_amplitude:  1


model filter: remove bleaching or trends: 100%|███████████████████████████████████████████████████████████████████| 1232/1232 [00:01<00:00, 907.36it/s]
binarizing continuous traces filtered fluorescence onphase: 100%|█████████████████████████████████████████████████| 1232/1232 [00:02<00:00, 490.16it/s]
binarizing continuous traces filtered fluorescence upphase: 100%|█████████████████████████████████████████████████| 1232/1232 [00:05<00:00, 244.51it/s]


   Oasis based binarization skipped by default ... 


# Visualize results - single cells

In [ ]:
# Next we turn off the flag that recomputes binarization - because we arleady did that
c.recompute_binarization = False

# ... and load our binarization data
c.load_binarization()


In [39]:
# pick a random cell to visuzlie
cell_id = 0

# scale value for matching height of 
scale = 100

# 
#fig=plt.figure(figsize=(20,20))
c.plot_cell_binarization_and_ca(cell_id, scale)


In [ ]:
# TASK: Spend some time visualizing lots of cells from early in the stack, i.e. cells 0,1,2...; middle 500,600, and end of the stack like 1000

# Question what do you notice about the differences between cells?


# Visualize results: rasters of all cells

In [40]:
# finally we now want to visualize the location of the 

# select some cell ids:
cell_ids = [0, 314, 501, 989] # can have any combination of cells you want
                              # for example: 
                              #    cell_ids = np.arange(0,1000, 100) 
                              #  picks every 100th cell from 0 to 1000

#
c.plot_cell_binarization_multi_cell(cell_ids)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [38]:
# 
c.show_rasters()
